In [1]:

import cv2
import matplotlib

import dlib
from PIL import Image
import pandas as pd
from matplotlib import pyplot as plt
cap = cv2.VideoCapture(0)

hog_face_detector = dlib.get_frontal_face_detector()

dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [2]:
def get_cropped_image_if_mouth(image_path):
    
    frame=plt.imread(image_path)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = hog_face_detector(gray)
    for face in faces:
        x1=face.left()
        y1=face.top()
        x2=face.right()
        y2=face.bottom()
        
        face_img=cv2.rectangle(frame, (x1,y1), (x2,y2),(0,255,0),1)
        face_landmarks = dlib_facelandmark(gray, face)
        
        X1=face_landmarks.part(48).x
        Y1=face_landmarks.part(33).y
        X2=face_landmarks.part(54).x
        Y2=face_landmarks.part(57).y
       
        #roi_color = face_img[y1:y2, x1:x2]
        roi = face_img[Y1:Y2, X1:X2]

        return roi

In [3]:
def transform(roi):
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    IMG_SIZ=224
    roi= cv2.resize(roi,(IMG_SIZ,IMG_SIZ),3)
    return roi

In [29]:
path_to_data = "E:\Test"
path_to_cr_data = "E:\Test\cropped"

In [30]:
import os
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [31]:
img_dirs

['E:\\Test\\test', 'E:\\Test\\train']

In [32]:
import shutil
if os.path.exists(path_to_cr_data):
     shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

In [33]:
from shutil import copy
cropped_image_dirs = []
folder_names_dict = {}

for img_dir in img_dirs:
    count = 1
    folder_name = img_dir.split('\\')[-1]
    print(folder_name)
    
    folder_names_dict[folder_name] = []
    
    for entry in os.scandir(img_dir):
        roi = get_cropped_image_if_mouth(entry.path)
        if roi is not None:
            roi=transform(roi)
            cropped_folder = path_to_cr_data + folder_name
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_image_dirs.append(cropped_folder)
                print("Generating cropped images in folder: ",cropped_folder)
                
            cropped_file_name = folder_name + str(count) + ".jpg"
            cropped_file_path = cropped_folder  + "/" + cropped_file_name 

            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            Image.fromarray(roi).save(cropped_file_path)
            folder_names_dict[folder_name].append(cropped_file_path)
            count += 1
            print(count)

test


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Test\\croppedtest/test1.jpg'